## Construindo nosso manipulador

Para a construção do nosso manipulador, inicializaremos ele na posição padrão.

In [5]:
from src.robot import Manipulator, Joint

In [6]:
joint1 = Joint(140, 0, 100, 90)
joint2 = Joint(0, 50, 0, 0, link=joint1)
joint3 = Joint(0, 50, -90, 0, link=joint2)

In [7]:
robot = Manipulator([joint1, joint2, joint3])

In [8]:
robot.forward_kinematics()
robot.static_plot()

In [9]:
robot.get_X(), robot.get_Y(), robot.get_Z()

(-8.682408883346513, 49.2403876506104, 90.0)

In [10]:
robot.update_theta(0, 105)
robot.update_theta(1, 6)
robot.update_theta(2, -90)


robot.forward_kinematics()
robot.static_plot()

In [11]:
robot.get_X(), robot.get_Y(), robot.get_Z()

(-14.22275821708668, 53.08005628993523, 95.50032839496899)

## Gerando nossos dados de treino

Para gerar nossos dados de treino, passaremos uma série de valores de $\theta$ para nosso manipulador e salvaremos os valores das coordenadas do endeffector. Assim, na hora de treinar, nosso input será as coordenadas do endeffector e nosso output os valores de theta.

In [12]:
from src.data.make_dataset import make_dataset

In [13]:
theta1_zone1 = list(range(0, 90, 10)) + list(range(180, 270, 10))
theta1_zone2 = list(range(90, 180, 10)) + list(range(270, 360, 10))
theta2 = list(range(0, 90,10))
theta3 = list(range(-90, 0, 10))

In [14]:
X_zone1, y_zone1 = make_dataset(robot, [theta1_zone1, theta2, theta3], 'data_zone1')
X_zone2, y_zone2 = make_dataset(robot, [theta1_zone2, theta2, theta3], 'data_zone2')

In [15]:
X_zone2[324], y_zone2[324]

(array([-32.13938048,  38.30222216,  90.        ]), array([130.,   0., -90.]))

### Visualização da área de trabalho gerada

In [16]:
theta1 = list(range(0, 360, 10))

In [17]:
robot.plot_workspace([theta1, theta2, theta3])

## Treinamento da rede neural

### Pré-processamento dos dados

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
indices = np.arange(X.shape[0])
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

In [ ]:
X, y

In [ ]:
# scaler_y = MinMaxScaler()
scaler_X = MinMaxScaler()
X = scaler_X.fit_transform(X)
# y = scaler_y.fit_transform(y)

### Definindo topologia da rede

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Activation, LeakyReLU, Input, Lambda, Concatenate, Dropout

In [ ]:
# model = keras.Sequential()
# model.add(layers.Dense(100, activation="tanh", input_shape=(None, 6)))
model2 = keras.Sequential()
model2.add(layers.Dense(100, input_dim=3, activation='relu'))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(32, activation='relu'))
model2.add(layers.Dense(3,))
# model.add(layers.Dense(3, activation="linear"))

In [ ]:
model2 = keras.Sequential([
    Dense(256, input_shape=(3,)),
    LeakyReLU(),
    Dense(256),
    LeakyReLU(),
    Dropout(0.5), 
    Dense(256),
    LeakyReLU(),
    Dense(256),
    LeakyReLU(),
    Dense(256),
    LeakyReLU(),
    Dropout(0.5), 
    Dense(256),
    LeakyReLU(),
    Dense(256),
    LeakyReLU(),
    Dense(3)
])

In [ ]:
def teste(output):
    from matriz_homogenea import Manipulator, Joint
    joint1 = Joint(140, 0, 100, 90)
    joint2 = Joint(0, 50, 0, 0, link=joint1)
    joint3 = Joint(0, 50, -90, 0, link=joint2)
    robot = Manipulator([joint1, joint2, joint3])
    output_true = []
    for t1_true, t2_true, t3_true in output:
        robot.update_theta(0, t1_true)
        robot.update_theta(1, t2_true)
        robot.update_theta(2, t3_true)
        robot.forward_kinematics()
        output_true.append([robot.get_X(), robot.get_Y(), robot.get_Z()])
    return np.array(output_true)

In [ ]:
def transform_matrix_tensor(theta, d, a, alpha):
    # tensor version of transform matrix
    matrix = [[tf.cos(theta), tf.multiply(-tf.sin(theta), tf.cos(alpha)), tf.multiply(tf.sin(theta), tf.sin(alpha)), tf.multiply(a, tf.cos(theta))], 
              [tf.sin(theta), tf.multiply(tf.cos(theta), tf.cos(alpha)), tf.multiply(-tf.cos(theta), tf.sin(alpha)), tf.multiply(a, tf.sin(theta))], 
              [tf.zeros_like(theta), tf.sin(alpha), tf.cos(alpha), d], 
              [tf.zeros_like(theta), tf.zeros_like(theta), tf.zeros_like(theta), tf.ones_like(theta)]]
    return matrix

def batch_matmul(location_v, batch_theta_v, d, a, alpha):
    # perform matrix multiplication between the location vector and the transform matrix, 
    # independently for each example in the batch, but done in a parallel way
    d = tf.math.scalar_mul(d, tf.ones_like(batch_theta_v))
    alpha = tf.math.scalar_mul(alpha, tf.ones_like(batch_theta_v))
    a = tf.math.scalar_mul(a, tf.ones_like(batch_theta_v))
    m0 = transform_matrix_tensor(batch_theta_v, d, a, alpha)
    m = tf.multiply(m0, location_v)
    m = tf.reduce_sum(m, axis=1)
    m = tf.transpose(m)
    return m

def forward_kinematics_loss_3(y_true, y_pred):
    # y_true is the xy position
    # y_pred is the 2-dimensional theta output
    theta1 = y_pred[:, 0]
    theta2 = y_pred[:, 1]
    theta3 = y_pred[:, 2]
    zeros = tf.zeros_like(theta1)
    zeros = K.expand_dims(zeros, axis=1)
    
    location_v = K.concatenate([zeros, zeros, zeros, zeros+1], axis=1)
    location_v = K.expand_dims(location_v, axis=-1)
    location_v = K.concatenate([location_v]*4, axis=2)
    location_v = tf.transpose(location_v, perm=[2, 1, 0])
    
    end_tip_1st_segment = batch_matmul(location_v, theta1, 140., 0., 90.)
    location_v = K.expand_dims(end_tip_1st_segment, axis=-1)
    location_v = K.concatenate([location_v]*4, axis=2)
    location_v = tf.transpose(location_v, perm=[2, 1, 0])
    
    end_tip_2nd_segment = batch_matmul(location_v, theta2, 0., 50., 0.)
    
    location_v = K.expand_dims(end_tip_2nd_segment, axis=-1)
    location_v = K.concatenate([location_v]*4, axis=2)
    location_v = tf.transpose(location_v, perm=[2, 1, 0])
    
    end_tip_3nd_segment = batch_matmul(location_v, theta3, 0., 50., 0.)
    
    xyz = end_tip_3nd_segment[:, :3]
    loss = K.mean(K.square(xyz - y_true))
    return loss

In [ ]:
@tf.function(autograph=False)
def iterate_tf(t, robot):
    output = []
    for t1_true, t2_true, t3_true in t.numpy():
            robot.update_theta(0, t1_true)
            robot.update_theta(1, t2_true)
            robot.update_theta(2, t3_true)
            robot.forward_kinematics()
            output.append([robot.get_X(), robot.get_Y(), robot.get_Z()])
    return np.array(output)

def custom_loss(y_true, y_pred):
    from matriz_homogenea import Manipulator, Joint
    joint1 = Joint(140, 0, 100, 90)
    joint2 = Joint(0, 50, 0, 0, link=joint1)
    joint3 = Joint(0, 50, -90, 0, link=joint2)
    robot = Manipulator([joint1, joint2, joint3])
    output_true = []
    output_pred = []
    output_true = iterate_tf(y_true, robot)
    output_pred = iterate_tf(y_pred, robot)
    return tf.keras.losses.mean_squared_error(output_true,output_pred)

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=1000,
    decay_rate=0.9)

model2.compile(optimizer = keras.optimizers.Adam(lr=0.1), loss=forward_kinematics_loss_3, run_eagerly=True)

In [ ]:
model2.summary()

In [ ]:
import keras.backend as K

In [ ]:
history = model2.fit(X, X, epochs=100000, batch_size=32, workers=-1,
    use_multiprocessing=True)

In [ ]:
model.save("first_experiment")

In [ ]:
output = model2.predict(scaler_X.transform([[-8.682408883346513, 49.2403876506104, 90.0]]))
output

## Trejetória de teste

In [ ]:
X, Y, Z = [], [], []
input_pred = []
final_X, final_Y, final_Z = [], [], []
points = []
robot.update_theta(0, 0)
robot.update_theta(1, 0)
robot.update_theta(2, -90)
theta2 = 0
theta3 = -90
for k in range(3):
    for i in range(0, 360, 15):
        robot.update_theta(0, i)
        robot.update_theta(1, theta2)
        robot.update_theta(2, theta3)
#         robot.update_theta(2, k)
        robot.forward_kinematics()
        X = robot.get_all_X()
        Y = robot.get_all_Y()
        Z = robot.get_all_Z()
        points.append([X, Y, Z])
        input_pred.append([robot.get_X(), robot.get_Y(), robot.get_Z()])
        final_X.append(robot.get_X())
        final_Y.append(robot.get_Y())
        final_Z.append(robot.get_Z())
        theta2 += 1.25
        theta3 += 1.25

In [ ]:
X_2, Y_2, Z_2 = [], [], []
input_pred_2 = []
final_X_2, final_Y_2, final_Z_2 = [], [], []
points_2 = []
robot.update_theta(0, 100)
robot.update_theta(1, 0)
robot.update_theta(2, -90)
theta2 = 0
theta3 = -90
for i in range(-90, 90, 10):
    robot.update_theta(2, i)
#         robot.update_theta(2, k)
    robot.forward_kinematics()
    X = robot.get_all_X()
    Y = robot.get_all_Y()
    Z = robot.get_all_Z()
    points_2.append([X, Y, Z])
    input_pred_2.append([robot.get_X(), robot.get_Y(), robot.get_Z()])
    final_X_2.append(robot.get_X())
    final_Y_2.append(robot.get_Y())
    final_Z_2.append(robot.get_Z())

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x=final_X, y=final_Y, z=final_Z,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
))

fig.update_layout(scene = dict(
                                xaxis = dict(nticks=8, range=[min(final_X) - 1,max(final_X) + 1]),
                                yaxis = dict(nticks=8, range=[min(final_Y) - 1,max(final_Y) + 1]),
                                zaxis = dict(nticks=8, range=[min(final_Z) - 1,max(final_Z) + 1])))
fig.show()

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x=final_X_2, y=final_Y_2, z=final_Z_2,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
))

fig.update_layout(scene = dict(
                                xaxis = dict(nticks=8, range=[min(final_X_2) - 1,max(final_X_2) + 1]),
                                yaxis = dict(nticks=8, range=[min(final_Y_2) - 1,max(final_Y_2) + 1]),
                                zaxis = dict(nticks=8, range=[min(final_Z_2) - 1,max(final_Z_2) + 1])))
fig.show()

In [ ]:
output = []
for X, Y, Z in input_pred:
    if (X >= 0 and Y >= 0) or (X < 0 and Y < 0):
        output.append(model.predict(scaler_X.transform([[X, Y, Z]]))[0])
    else:
        output.append(model2.predict(scaler_X.transform([[X, Y, Z]]))[0])

        
final_points = []
final_X_, final_Y_, final_Z_ = [], [], []
for theta1, theta2, theta3 in output:
    robot.update_theta(0, theta1)
    robot.update_theta(1, theta2)
    robot.update_theta(2, theta3)
    robot.forward_kinematics()
    final_X_.append(robot.get_X())
    final_Y_.append(robot.get_Y())
    final_Z_.append(robot.get_Z())
    final_points.append([robot.get_X(), robot.get_Y(), robot.get_Z()])

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x=final_X_, y=final_Y_, z=final_Z_,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='orange',
        width=2
    )
))

fig.add_trace(
go.Scatter3d(
    x=final_X, y=final_Y, z=final_Z,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
))

fig.update_layout(scene = dict(
                                 xaxis = dict(nticks=8, range=[-110,110]),
                                yaxis = dict(nticks=8, range=[-110, 110]),
                                zaxis = dict(nticks=8, range=[0, 250])))
fig.show()

In [ ]:
output = []
for X, Y, Z in input_pred_2:
    if (X >= 0 and Y >= 0) or (X < 0 and Y < 0):
        output.append(model.predict(scaler_X.transform([[X, Y, Z]]))[0])
    else:
        output.append(model2.predict(scaler_X.transform([[X, Y, Z]]))[0])

points_final = []
final_X_, final_Y_, final_Z_ = [], [], []
for theta1, theta2, theta3 in output:
    robot.update_theta(0, theta1)
    robot.update_theta(1, theta2)
    robot.update_theta(2, theta3)
    robot.forward_kinematics()
    final_X_.append(robot.get_X())
    final_Y_.append(robot.get_Y())
    final_Z_.append(robot.get_Z())
    points_final.append([robot.get_X(), robot.get_Y(), robot.get_Z()])

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x=final_X_, y=final_Y_, z=final_Z_,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='orange',
        width=2
    )
))

fig.add_trace(
go.Scatter3d(
    x=final_X_2, y=final_Y_2, z=final_Z_2,
    marker=dict(
        size=0,
        color=3,
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
))
fig.update_layout(scene = dict(
                                xaxis = dict(nticks=8, range=[-110,110]),
                                yaxis = dict(nticks=8, range=[-110, 110]),
                                zaxis = dict(nticks=8, range=[0, 200])))
fig.show()

In [ ]:
points = np.array([[100, 0, i] for i in range(-90, 90, 10)])
tf.keras.losses.mean_squared_error(points,np.array(output))

In [ ]:
np.mean(np.square(points - np.array(output)), axis=-1)

In [ ]:
np.mean(np.abs(final_points - np.array(input_pred)))